In [206]:
import pandas as pd

In [211]:


# Carregar o arquivo CSV
df = pd.read_csv("EURUSD_M1.csv")  # Certifique-se de substituir pelo caminho correto

# Renomear as colunas para facilitar a manipulação
df.columns = ['data', 'Preço de abertura', 'Preço Máximo', 'Preço Mínimo', 'Preço de fechamento', 'Volume']

# Converter a coluna 'data' para datetime, caso não tenha sido convertido ainda
df["data"] = pd.to_datetime(df["data"], errors='coerce')  # 'errors="coerce"' converte valores inválidos em NaT

df_filtrado = df[~df['data'].dt.year.isin(range(2017, 2024))]

# Extraia o horário da coluna 'data' para facilitar o filtro
df_filtrado["hora"] = df_filtrado["data"].dt.time  # Formato de hora: "HH:MM:SS"

# Crie duas condições para horários >= 23:00 OU < 03:00
filtro_horario = (df_filtrado["hora"] >= pd.to_datetime("10:00:00").time()) | (df_filtrado["hora"] < pd.to_datetime("12:00:00").time())

# Filtrar os dados com base no horário
df_filtrado = df_filtrado[filtro_horario]

# Adicionar uma coluna "dia" para agrupar por dias diferentes
df_filtrado["dia"] = df_filtrado["data"].dt.date

# Agrupar os dados por dia e calcular estatísticas do período
resultado = df_filtrado.groupby("dia").agg({
    "Preço de abertura": "first",
    "Preço de fechamento": "last",
    "Preço Máximo": "max",
    "Preço Mínimo": "min",
    "Volume": "sum"
}).reset_index()

# Exibir o resultado
print(resultado)


            dia  Preço de abertura  Preço de fechamento  Preço Máximo  Preço Mínimo  Volume
0    2024-07-12            1.09015              1.09054       1.09113       1.09010   19567
1    2024-07-15            1.08917              1.08909       1.09222       1.08837  224596
2    2024-07-16            1.08908              1.09042       1.09068       1.08715  214916
3    2024-07-17            1.09043              1.09350       1.09480       1.08950  253849
4    2024-07-18            1.09350              1.08920       1.09388       1.08863  226971
..          ...                ...                  ...           ...           ...     ...
137  2025-01-21            1.03877              1.04136       1.04353       1.03420  175993
138  2025-01-22            1.04136              1.04035       1.04573       1.03967  174932
139  2025-01-23            1.04035              1.04486       1.04489       1.03719  188091
140  2025-01-24            1.04485              1.04919       1.05213       1.04

In [208]:
# Função para classificar o período como "Compra" ou "Venda"
def classificar_periodo(row):
    if row["Preço de fechamento"] < row["Preço de abertura"]:
        return "Compra"
    elif row["Preço de fechamento"] > row["Preço de abertura"]:
        return "Venda"
    else:
        return "Neutro"  # Caso os preços sejam iguais

# Adicionar a nova coluna com a classificação
resultado["Classificação"] = resultado.apply(classificar_periodo, axis=1)

# Exibir o DataFrame atualizado
print(resultado)


            dia  Preço de abertura  Preço de fechamento  Preço Máximo  Preço Mínimo  Volume Classificação
0    2024-07-12            1.09015              1.09054       1.09113       1.09010   19567         Venda
1    2024-07-15            1.08917              1.08909       1.09222       1.08837  224596        Compra
2    2024-07-16            1.08908              1.09042       1.09068       1.08715  214916         Venda
3    2024-07-17            1.09043              1.09350       1.09480       1.08950  253849         Venda
4    2024-07-18            1.09350              1.08920       1.09388       1.08863  226971        Compra
..          ...                ...                  ...           ...           ...     ...           ...
137  2025-01-21            1.03877              1.04136       1.04353       1.03420  175993         Venda
138  2025-01-22            1.04136              1.04035       1.04573       1.03967  174932        Compra
139  2025-01-23            1.04035            

In [209]:
contagem = resultado['Classificação'].value_counts()

In [210]:
porcentagem = (contagem / contagem.sum()) * 100
print(porcentagem)

Classificação
Compra    58.450704
Venda     41.549296
Name: count, dtype: float64
